In [123]:
import configparser
import requests
from requests import Session
from psycopg2 import connect
from psycopg2.extras import execute_values
from time import sleep
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\nchan6\Documents\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [214]:
def get_url(mapserver, id):
    "This prints the base_url for the request"
    url = "https://insideto-gis.toronto.ca/arcgis/rest/services/{}/MapServer/{}/query".format(mapserver, id)
    return(url)

In [516]:
def check_status(max_number, record_max, output_table, insert_column):
    "This gets the request status"
    try:
        query = {"where": "1=1", 
                "outFields": "*",
                "outSR": '4326',         
                "returnGeometry": "true",
                "returnTrueCurves": "false",
                "returnIdsOnly": "false",
                "returnCountOnly": "false",
                "returnZ": "false",
                "returnM": "false",
                "returnDistinctValues": "false",
                "resultOffset": "{}".format(max_number),
                "resultRecordCount": "{}".format(record_max),
                "orderByFields": "OBJECTID", 
                "returnExtentsOnly": "false",
                "f":"json"}  

        print(query)
        print(max_number, record_max)
        r = requests.get(base_url, params = query, verify = False)
    except requests.exceptions.SSLError:
        sleep(5)
        error_status = 'somethings up'
        rule = 'dont add'
        status = (error_status, rule)
    else:
        return_json = r.json()
        features = return_json['features']
        fields = return_json['fields']
        keys = [field['name'] for field in fields]
        rows = []
        for feature in features:
            geometry = 'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')'
            row = [feature['attributes'][key] for key in keys]
            row.append(geometry)
            rows.append(row)          

        sql='INSERT INTO {} {} VALUES %s'.format(output_table, insert_column)

        with con:
            with con.cursor() as cur:
                execute_values(cur, sql, rows)   
                
        if return_json.get('exceededTransferLimit', False) == True:
            rule = 'add'
        else:
            rule = 'dont add'  
            
        error_status = 'success'
        status = (error_status, rule)    
    return status

In [517]:
def get_layer(mapserver, id, output_table):
    
    """
    This function calls to the GCCview rest API and inserts the outputs to the output table in the postgres database.

    Parameters
    ----------
    mapserver : string
        The name of the mapserver that host the desire layer

    id : int
        The id of desire layer

    output_table: string
        Name of table that returned row will be inserted into
    """  
    base_url = get_url({}, {}).format(mapserver, id)
    query = {"where":"1=1",
             "outFields": "*",
             "outSR": '4326',         
             "returnGeometry": "true",
             "returnTrueCurves": "false",
             "returnIdsOnly": "false",
             "returnCountOnly": "false",
             "returnZ": "false",
             "returnM": "false",
             "orderByFields": "OBJECTID", 
             "returnDistinctValues": "false",
             "returnExtentsOnly": "false",
             "f":"json"}
    
    # First get request to get maxnumber and append first request
    r = requests.get(base_url, params = query, verify = False)
    return_json = r.json()
    features = return_json['features']
    record_max=(len(features))
    max_number = record_max
    rows = []
    
    # Create a table in postgres based on returned fields
    fields = return_json['fields']
    new_column = '('
    insert_column= '('
    for field in fields:
        column_name = (field['name'].lower()).replace('.', '_')
        if field['type'] == 'esriFieldTypeInteger' or field['type'] == 'esriFieldTypeInteger' or field['type'] =='esriFieldTypeOID' or field['type'] == 'esriFieldTypeSmallInteger' or field['type'] =='esriFieldGlobalID':
                column_type = 'integer'
        elif field['type'] == 'esriFieldTypeString':
                column_type = 'text'
        elif field['type'] == 'esriFieldTypeDouble':
                column_type = 'numeric'
        elif field['type'] == 'esriFieldTypeDate':
                column_type = 'date'
                               
        new_column = new_column + column_name +' '+column_type+', '
        insert_column = insert_column + column_name +','
        
    new_column = new_column +'geom geometry)' 
    insert_column = insert_column + 'geom)'
    
    sql= '''create table {} {}'''.format(output_table, new_column)
    
    with con:
        with con.cursor() as cur:
            cur.execute(sql)  
    
    # We first append the first set of features    
    keys = [field['name'] for field in fields]
    rows = []
    for feature in features:
        geometry = 'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')'
        row = [feature['attributes'][key] for key in keys]
        row.append(geometry)
        rows.append(row)          
    
    sql='INSERT INTO {} {} VALUES %s'.format(output_table, insert_column)
        
    with con:
        with con.cursor() as cur:
            execute_values(cur, sql, rows)        
    
    if return_json.get('exceededTransferLimit', False) == True:
        error_status = 'successful'
        rule = 'add'
    else:
        error_status = 'successful'
        rule = 'dont add'
    status = (error_status, rule)
    
    print(status)
    print(max_number, record_max)
    if status[1] == 'add':
        while status[0] =='successful' or status[0] == 'somethings up' or status[1] == 'add':

            status = check_status(max_number, record_max, output_table, insert_column)

            if status[0] == 'successful' or status[1] == 'add':
                max_number = max_number + record_max
            elif status[0] == 'somethings up' or status[1] == 'dont add':
                continue 
                
            print(max_number)
    else:
        print('all rows inserted')

In [518]:
get_layer('cot_geospatial2', 2, 'gcc_table')

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


('successful', 'add')
1000 1000
{'where': '1=1', 'outFields': '*', 'outSR': '4326', 'returnGeometry': 'true', 'returnTrueCurves': 'false', 'returnIdsOnly': 'false', 'returnCountOnly': 'false', 'returnZ': 'false', 'returnM': 'false', 'returnDistinctValues': 'false', 'resultOffset': '1000', 'resultRecordCount': '1000', 'orderByFields': 'OBJECTID', 'returnExtentsOnly': 'false', 'f': 'json'}
1000 1000


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2000
{'where': '1=1', 'outFields': '*', 'outSR': '4326', 'returnGeometry': 'true', 'returnTrueCurves': 'false', 'returnIdsOnly': 'false', 'returnCountOnly': 'false', 'returnZ': 'false', 'returnM': 'false', 'returnDistinctValues': 'false', 'resultOffset': '2000', 'resultRecordCount': '1000', 'orderByFields': 'OBJECTID', 'returnExtentsOnly': 'false', 'f': 'json'}
2000 1000


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


6000
{'where': '1=1', 'outFields': '*', 'outSR': '4326', 'returnGeometry': 'true', 'returnTrueCurves': 'false', 'returnIdsOnly': 'false', 'returnCountOnly': 'false', 'returnZ': 'false', 'returnM': 'false', 'returnDistinctValues': 'false', 'resultOffset': '6000', 'resultRecordCount': '1000', 'orderByFields': 'OBJECTID', 'returnExtentsOnly': 'false', 'f': 'json'}
6000 1000


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


7000
{'where': '1=1', 'outFields': '*', 'outSR': '4326', 'returnGeometry': 'true', 'returnTrueCurves': 'false', 'returnIdsOnly': 'false', 'returnCountOnly': 'false', 'returnZ': 'false', 'returnM': 'false', 'returnDistinctValues': 'false', 'resultOffset': '7000', 'resultRecordCount': '1000', 'orderByFields': 'OBJECTID', 'returnExtentsOnly': 'false', 'f': 'json'}
7000 1000


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
